In [22]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from keras.models import model_from_json

import warnings
warnings.filterwarnings("ignore")

# My seed

seed = 42

### Loading the training and test dataset

In [2]:
df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [4]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


### Spliting the dataset

In [5]:
X_train = df_train.drop(['label'], axis=1)
y_train = df_train['label']
X_test = df_test

# Free memory space

del df_train
del df_test

print('Shape of X_train:', X_train.shape)
print('Shape of y_train:', y_train.shape)
print('Shape of X_test :', X_test.shape)

Shape of X_train: (42000, 784)
Shape of y_train: (42000,)
Shape of X_test : (28000, 784)


### Counting the labels

In [6]:
counter = Counter(y_train)
counter

Counter({1: 4684,
         0: 4132,
         4: 4072,
         7: 4401,
         3: 4351,
         5: 3795,
         8: 4063,
         9: 4188,
         2: 4177,
         6: 4137})

### Normalizing the values of training and test

In [7]:
X_train = X_train / 255
X_test = X_test / 255

### Reshape the images in 3 dimensions to use with Keras

In [8]:
X_train = X_train.values.reshape(-1,28,28,1) # (height = 28px, width = 28px , canal = 1)
X_test = X_test.values.reshape(-1,28,28,1)

print('Shape of X_train:', X_train.shape)
print('Shape of X_test :', X_test.shape)

Shape of X_train: (42000, 28, 28, 1)
Shape of X_test : (28000, 28, 28, 1)


### Converting y values (labels) to categorical values

In [9]:
# One Hot Categories
y_train = to_categorical(y_train, num_classes = 10)
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

### Define the baseline neural network model

In [10]:
def baseline_model():
    
    # Create baseline
    
    baseline = Sequential()

    #---------------------------------------------------------------------------------------------------
    
    # 32 filters for the two firsts conv2D layers
    
    baseline.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', 
                     input_shape = (28, 28, 1)))
    baseline.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    
    # This layer simply acts as a downsampling filter. 
    # It looks at the 2 neighboring pixels and picks the maximal value, reducing computational cost, 
    # and to some extent also reduce overfitting.
    
    # IMPORTANT: Combining convolutional and pooling layers, CNN are able to combine local features and 
    # learn more global features of the image.
    
    baseline.add(MaxPool2D(pool_size=(2,2)))
    
    # Dropout is a regularization method, where a proportion of nodes (25%) in the layer are randomly ignored 
    # for each training sample. This dropout forces the network to learn features in a distributed way 
    # and improves generalization and reduces the overfitting.
    
    baseline.add(Dropout(0.25))
    #---------------------------------------------------------------------------------------------------
    
    # 64 filters for the two last conv2D layers
    
    baseline.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
    baseline.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu'))
    
    baseline.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    baseline.add(Dropout(0.25))
    #---------------------------------------------------------------------------------------------------

    # The Flatten layer is use to convert the final feature maps into a one single 1D vector. 
    # IMPORTANT: It combines all the found local features of the previous convolutional layers.
    
    baseline.add(Flatten())
    baseline.add(Dense(256, activation = "relu"))
    baseline.add(Dropout(0.5))
    
    # The net outputs distribution of probability of each class --> In our case, 10 output classes
    
    baseline.add(Dense(10, activation = "softmax"))
    
    # The optimizer will iteratively improve parameters in order to minimize the loss.
    
    optimizer = RMSprop(epsilon=1e-08)

    # Compile the baseline including the optimizer and evaluating the performance of the baseline by accuracy
    
    baseline.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
    
    return baseline

### Learning Rate

In [11]:
# If after the third epoch we didn't have an improvement of accuracy, the learning rate will be 
# reduced by 50% (factor).

lr_reduction = ReduceLROnPlateau(monitor='val_acc',
                                 patience=3, 
                                 verbose=0, 
                                 factor=0.5, 
                                 min_lr=0.00001)

### Data augmentation

In [12]:
# The idea is to alter the training data with small transformations to reproduce the variations 
# occuring when someone is writing a digit. It's a way to minimize the overfitting of the model.

generator = ImageDataGenerator(featurewise_center = False,
                               samplewise_center = False, 
                               featurewise_std_normalization = False,
                               samplewise_std_normalization = False,
                               zca_whitening = False,
                               rotation_range = 15, # Rotate image in 15 degrees
                               zoom_range = 0.15, # Zoom image (15% zoom) 
                               width_shift_range = 0.15, # Shift image horizontally (15% of width)
                               height_shift_range = 0.15, # Shift image vertically (15% of height)
                               horizontal_flip = False,
                               vertical_flip = False)

generator.fit(X_train)

### Creating 10 nets and training every ones

In [13]:
nets = 10
digits = [0] * nets
history = [0] * nets

epochs = 45
batch_size = 90

In [14]:
for model in range(nets):
    digits[model] = baseline_model()
    
    # Splitting train and test datasets
    
    X_train_aux, X_test_aux, y_train_aux, y_test_aux = train_test_split(X_train, y_train, test_size = 0.1)
    
    history[model] = digits[model].fit_generator(generator.flow(X_train_aux,
                                                              y_train_aux, 
                                                              batch_size = batch_size),
                                                 epochs = epochs, 
                                                 steps_per_epoch = X_train_aux.shape[0] // batch_size, 
                                                 validation_data = (X_test_aux, y_test_aux), 
                                                 callbacks=[lr_reduction],
                                                 verbose=1)
    
    print("CNN Model {0:d}: Epochs = {1:d}, Train accuracy = {2:.5f}, Validation accuracy = {3:.5f}".format(
        model + 1, # Number of the CNN model
        epochs, # Total of epochs
        max(history[model].history['acc']), # Maximum Accuracy from Training
        max(history[model].history['val_acc']))) # Maximum Accuracy from Test (validation)

Epoch 1/45
420/420 [==============================] - 237s 564ms/step - loss: 0.6024 - acc: 0.8051 - val_loss: 0.1117 - val_acc: 0.9664
Epoch 2/45
420/420 [==============================] - 236s 562ms/step - loss: 0.1798 - acc: 0.9462 - val_loss: 0.0520 - val_acc: 0.9821
Epoch 3/45
420/420 [==============================] - 236s 561ms/step - loss: 0.1239 - acc: 0.9628 - val_loss: 0.0447 - val_acc: 0.9867
Epoch 4/45
420/420 [==============================] - 235s 560ms/step - loss: 0.1029 - acc: 0.9698 - val_loss: 0.0315 - val_acc: 0.9883
Epoch 5/45
420/420 [==============================] - 235s 559ms/step - loss: 0.0897 - acc: 0.9738 - val_loss: 0.0425 - val_acc: 0.9888
Epoch 6/45
420/420 [==============================] - 235s 561ms/step - loss: 0.0856 - acc: 0.9756 - val_loss: 0.0355 - val_acc: 0.9893
Epoch 7/45
420/420 [==============================] - 236s 561ms/step - loss: 0.0806 - acc: 0.9762 - val_loss: 0.0426 - val_acc: 0.9902
Epoch 8/45
420/420 [============================

420/420 [==============================] - 244s 582ms/step - loss: 0.0588 - acc: 0.9828 - val_loss: 0.0207 - val_acc: 0.9929
Epoch 12/45
420/420 [==============================] - 241s 575ms/step - loss: 0.0612 - acc: 0.9829 - val_loss: 0.0235 - val_acc: 0.9929
Epoch 13/45
420/420 [==============================] - 241s 575ms/step - loss: 0.0564 - acc: 0.9838 - val_loss: 0.0190 - val_acc: 0.9950
Epoch 14/45
420/420 [==============================] - 243s 578ms/step - loss: 0.0572 - acc: 0.9838 - val_loss: 0.0222 - val_acc: 0.9921
Epoch 15/45
420/420 [==============================] - 242s 576ms/step - loss: 0.0584 - acc: 0.9839 - val_loss: 0.0220 - val_acc: 0.9936
Epoch 16/45
420/420 [==============================] - 241s 575ms/step - loss: 0.0610 - acc: 0.9836 - val_loss: 0.0217 - val_acc: 0.9940

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 17/45
420/420 [==============================] - 242s 575ms/step - loss: 0.0500 - acc: 0.9853 - val_los

420/420 [==============================] - 243s 578ms/step - loss: 0.0383 - acc: 0.9885 - val_loss: 0.0151 - val_acc: 0.9950
Epoch 29/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0407 - acc: 0.9887 - val_loss: 0.0148 - val_acc: 0.9950

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 30/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0365 - acc: 0.9892 - val_loss: 0.0157 - val_acc: 0.9948
Epoch 31/45
420/420 [==============================] - 242s 576ms/step - loss: 0.0384 - acc: 0.9890 - val_loss: 0.0153 - val_acc: 0.9952
Epoch 32/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0353 - acc: 0.9896 - val_loss: 0.0153 - val_acc: 0.9948
Epoch 33/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0382 - acc: 0.9889 - val_loss: 0.0154 - val_acc: 0.9948
Epoch 34/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0378 - acc: 0.9891 - val_loss: 0.0155 - val_

Epoch 38/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0382 - acc: 0.9889 - val_loss: 0.0209 - val_acc: 0.9940
Epoch 39/45
420/420 [==============================] - 243s 578ms/step - loss: 0.0364 - acc: 0.9894 - val_loss: 0.0204 - val_acc: 0.9945
Epoch 40/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0368 - acc: 0.9895 - val_loss: 0.0200 - val_acc: 0.9943
Epoch 41/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0382 - acc: 0.9892 - val_loss: 0.0200 - val_acc: 0.9945
Epoch 42/45
420/420 [==============================] - 242s 577ms/step - loss: 0.0366 - acc: 0.9895 - val_loss: 0.0199 - val_acc: 0.9945
Epoch 43/45
420/420 [==============================] - 243s 578ms/step - loss: 0.0365 - acc: 0.9898 - val_loss: 0.0197 - val_acc: 0.9945
Epoch 44/45
420/420 [==============================] - 242s 576ms/step - loss: 0.0375 - acc: 0.9893 - val_loss: 0.0199 - val_acc: 0.9945
Epoch 45/45
420/420 [====================

420/420 [==============================] - 244s 581ms/step - loss: 0.0776 - acc: 0.9801 - val_loss: 0.0381 - val_acc: 0.9912
Epoch 13/45
420/420 [==============================] - 245s 583ms/step - loss: 0.0765 - acc: 0.9797 - val_loss: 0.0322 - val_acc: 0.9924
Epoch 14/45
420/420 [==============================] - 243s 578ms/step - loss: 0.0791 - acc: 0.9796 - val_loss: 0.0317 - val_acc: 0.9917

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 15/45
420/420 [==============================] - 243s 580ms/step - loss: 0.0618 - acc: 0.9838 - val_loss: 0.0285 - val_acc: 0.9955
Epoch 16/45
420/420 [==============================] - 246s 585ms/step - loss: 0.0606 - acc: 0.9842 - val_loss: 0.0263 - val_acc: 0.9940
Epoch 17/45
420/420 [==============================] - 243s 579ms/step - loss: 0.0610 - acc: 0.9849 - val_loss: 0.0273 - val_acc: 0.9921
Epoch 18/45
420/420 [==============================] - 243s 579ms/step - loss: 0.0623 - acc: 0.9829 - val_los

Epoch 22/45
420/420 [==============================] - 244s 582ms/step - loss: 0.0603 - acc: 0.9833 - val_loss: 0.0293 - val_acc: 0.9933
Epoch 23/45
420/420 [==============================] - 243s 579ms/step - loss: 0.0583 - acc: 0.9846 - val_loss: 0.0336 - val_acc: 0.9931

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 24/45
420/420 [==============================] - 243s 580ms/step - loss: 0.0481 - acc: 0.9866 - val_loss: 0.0295 - val_acc: 0.9936
Epoch 25/45
420/420 [==============================] - 245s 583ms/step - loss: 0.0495 - acc: 0.9860 - val_loss: 0.0324 - val_acc: 0.9919
Epoch 26/45
420/420 [==============================] - 244s 581ms/step - loss: 0.0477 - acc: 0.9868 - val_loss: 0.0304 - val_acc: 0.9924

Epoch 00026: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 27/45
420/420 [==============================] - 247s 587ms/step - loss: 0.0442 - acc: 0.9880 - val_loss: 0.0272 - val_acc: 0.9933
Epoch 28/45
420/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



420/420 [==============================] - 237s 563ms/step - loss: 0.0412 - acc: 0.9883 - val_loss: 0.0258 - val_acc: 0.9924
Epoch 19/45
420/420 [==============================] - 241s 575ms/step - loss: 0.0404 - acc: 0.9881 - val_loss: 0.0264 - val_acc: 0.9914
Epoch 20/45
420/420 [==============================] - 244s 581ms/step - loss: 0.0407 - acc: 0.9884 - val_loss: 0.0241 - val_acc: 0.9921

Epoch 00020: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 21/45
420/420 [==============================] - 244s 581ms/step - loss: 0.0368 - acc: 0.9896 - val_loss: 0.0245 - val_acc: 0.9917
Epoch 22/45
420/420 [==============================] - 244s 582ms/step - loss: 0.0379 - acc: 0.9890 - val_loss: 0.0250 - val_acc: 0.9921
Epoch 23/45
420/420 [==============================] - 244s 581ms/step - loss: 0.0354 - acc: 0.9899 - val_loss: 0.0258 - val_acc: 0.9917

Epoch 00023: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 24/45
420/420 [========

### Getting the predictions with more probabilities to be correct

In [15]:
label_predicted = np.zeros( (X_test.shape[0], 10) ) 

for model in range(nets):
    label_predicted = label_predicted + digits[model].predict(X_test)
    
# Get the index with the maximum probability

label_predicted = np.argmax(label_predicted, axis = 1)
label_predicted = pd.Series(label_predicted, name = "Label")

In [16]:
solution = pd.concat([pd.Series(range(1, 28001), name = "ImageId"), label_predicted], axis = 1)
solution.to_csv("solution_cnn_v5.csv", index=False)

In [17]:
solution.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


### Saving and loading the models

In [19]:
for model in range(nets):
    model_saved = digits[model].to_json()
    name = 'model_' + str(model) + '.json'
    with open(name, 'w') as json_file:
        json_file.write(model_saved)
    name = 'model_' + str(model) + '.h5'
    digits[model].save_weights(name)

In [23]:
model_loaded = [0] * nets
for model in range(nets):
    name = 'model_' + str(model) + '.json'
    json_file = open(name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_loaded[model] = model_from_json(loaded_model_json)
    name = 'model_' + str(model) + '.h5'
    model_loaded[model].load_weights(name)